### Preparción



In [2]:
## Apertura de la conexión

import sqlite3
conn = sqlite3.connect(":memory:") 
cur = conn.cursor()

In [3]:
! tail -n 5 drivers.csv

'tail' is not recognized as an internal or external command,
operable program or batch file.


## Creación y carga de datos para la tabla drivers usando pandas

In [ ]:
import pandas as pd

drivers = pd.read_csv("drivers.csv",
                      sep = ',',           # separador de campos
                      thousands = None,    # separador de miles para números
                      decimal = '.',       # separador de los decimales para números
                      encoding='latin-1')

drivers.to_sql(name = "drivers",
              con = conn,
              if_exists = "replace")

In [4]:
cur.execute("SELECT * FROM drivers LIMIT 1;").fetchall()

OperationalError: no such table: drivers

### Creación y carga de datos de la tabla

In [ ]:
timesheet = pd.read_csv("timesheet.csv",
                       sep = ",",
                       thousands = None, ##Separador de miles para numeros
                        decimal = ".",
                        encoding = "latin-1")

timesheet.to_sql(name = "timesheet", #nombre de la tabla
                con = conn,   #conexión a la base de datos
                if_exists = "replace")
                    

In [ ]:
timesheet[0:1]

In [ ]:
cur.execute("SELECT driverId, week, hours_logged, miles_logged FROM timesheet LIMIT1; ").fetchall()


## Cantidad de horas y millas de cada conductor por año.

In [ ]:
cur.execute("""
SELECT 
    driverID, 
    sum(hours_logged),
    sum(miles_logged)
FROM 
    timesheet
GROUP BY 
    driverId;""").fetchall()

## Subconsultas

In [ ]:
##
## Selecciona todos los registros de la tabla timesheet
## que tienen el driverId igual al menor driverId de la
## tabla drivers.
##

cur.execute("""
SELECT
    *
FROM
    timesheet
WHERE
    driverId = (SELECT min(driverId) FROM drivers);
""").fetchall()

### Cosulta para unir las tablas

In [ ]:
cur.execute("""
SELECT
    d.driverId,
    d.name,
    t.total_hours,
    t.total_miles
FROM
    drivers d
JOIN (
    SELECT 
        driverId,
        sum(hours_logged)total_hours,
        sum(miles_logged)totalmiles
    FROM
        timesheet
    GROUP BY
        driverId
    )t
ON
    (d.driverId = t.driverId);
""").fetchall()

In [ ]:


cur.execute("""
SELECT
    d.driverId,
    d.name,
    t.total_hours,
    t.total_miles
FROM 
    drivers d
JOIN (
    SELECT
        driverId,
        sum(hours_logged)total_hours,
        sum(miles_logged)total_miles
    FROM
        timesheet
    GROUP BY
        driverId) t
ON
    (d.driverId = t.driverId)
ORDER BY
    t.total_hours DESC
LIMIT 1;
""").fetchall

## Creación de un DataFrame usando una consulta

In [6]:
df = pd.read_sql("""
SELECT
    d.driverId,
    d.name,
    t.total_hours,
    t.total_miles
FROM
    drivers d
JOIN (
    SELECT
        driverId,
        sum(hours_logged)total_hours,
        sum(miles_logged)total_miles
    FROM
        timesheet
    GROUP BY
        driverId
    )t
ON
    (d.driverId = t.driverId):
""", con = conn)

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
df.plot.bar(x="name", y="total_hours");

NameError: name 'pd' is not defined

## Almacenamiento de los resultados

In [ ]:
summary = cur.execute("""
SELECT
    d.driverId,
    d.name,
    t.total_hours,
    t.total_miles
FROM
    drivers d
JOIN (
    SELECT
        driverId,
        sum(hours_logged)total_hours,
        sum(miles_logged)total_miles
    FROM
        timesheet
    GROUP BY
        driverId
    ) t
ON
    (d.driverId = t.driverId)
ORDER BY
    d.name;
""").fetchall()


text = [[str(e) for e in row] for row in summary]
text = [",".join(row) for row in text]
text = "\n".join(text)

open("summary.csv", "wt").write(text)